# 🔍 Agentic RAG with Azure AI Foundry (Python)

## 📋 Learning Objectives

This notebook demonstrates how to build intelligent Retrieval-Augmented Generation (RAG) systems using the Microsoft Agent Framework with Azure AI Foundry. You'll learn to create agents that can search through documents and provide accurate, grounded responses based on uploaded content.

**Key RAG Capabilities You'll Build:**
- 📚 **Document Upload**: Upload and process documents for agent access
- 🔍 **Vector Store Creation**: Create searchable vector stores for semantic retrieval
- 🤖 **Grounded Responses**: Build agents that only answer from document content
- 🛡️ **Source Attribution**: Ensure responses are traceable to source documents

## 🎯 RAG Architecture Concepts

### Core RAG Components
- **Document Processing**: Upload files to Azure AI Foundry for indexing
- **Vector Store**: Semantic search capability for relevant content retrieval
- **File Search Tool**: Agent tool that queries the vector store
- **Grounded Generation**: AI responses based only on retrieved context

### Technical Flow
```python
Document Upload → Vector Store Creation → Agent Configuration
                        ↓                       ↓
User Query → File Search Tool → Retrieved Context → Grounded Response
```

## 🏗️ Technical Architecture

### Azure AI Foundry Integration
- **AIProjectClient**: Manages Azure AI project resources and operations
- **AzureAIAgentClient**: Provides agent functionality with Azure backend
- **Vector Store Management**: Handles document indexing and retrieval
- **File Search Tool**: Built-in tool for semantic document search

### Agent Behavior
- Only answers questions using information from uploaded documents
- Clearly indicates when questions cannot be answered from available content
- Provides explicit citations to source documents
- Avoids general knowledge responses outside document scope

## ⚙️ Prerequisites & Setup

**Required Dependencies:**
```bash
pip install agent-framework-core azure-ai-projects azure-identity -U
```

**Azure Configuration:**
- Azure subscription with AI Foundry access
- Azure CLI authentication configured
- Environment variables in `.env` file:
  ```env
  AZURE_AI_PROJECT_ENDPOINT=your_project_endpoint
  AZURE_AI_MODEL_DEPLOYMENT_NAME=your_model_deployment
  ```

**Authentication:**
```bash
az login --tenant <Your Azure Tenant>
```

## 📊 Use Cases

### Knowledge Management
- Corporate document Q&A systems
- Policy and procedure guidance
- Technical documentation assistants
- Training material search

### Customer Support
- Product documentation queries
- Troubleshooting guide search
- FAQ and knowledge base systems
- Support ticket assistance

## 🚀 Getting Started

Follow the cells below to:
1. Install required dependencies and import libraries
2. Create a vector store with sample documents
3. Configure an agent with file search capabilities
4. Query the agent and receive grounded responses

Let's build an intelligent document-aware assistant! 📖✨

In [1]:
# ! pip install -r ../../../Installation/requirements.txt -U

In [2]:
import os

from azure.identity.aio import AzureCliCredential
from azure.ai.agents.aio import AgentsClient
from dotenv import load_dotenv

from agent_framework import HostedFileSearchTool, Content
from agent_framework.azure import AzureAIProjectAgentProvider

## 📦 Import Required Libraries

Import Azure AI libraries for agent management, file operations, and the modern AzureAIProjectAgentProvider for simplified agent creation.

In [3]:
load_dotenv()

True

## 🔧 Load Environment Variables

Load Azure AI Foundry configuration from the `.env` file.

In [4]:
# Define the query for the agent
query = "What's GraphRAG?"

## ❓ Define Query

Set up the question to ask the RAG agent about the uploaded document.

In [5]:
file = None
vector_store = None

async with (
    AzureCliCredential() as credential,
    AgentsClient(endpoint=os.environ["AZURE_AI_PROJECT_ENDPOINT"], credential=credential) as agents_client,
    AzureAIProjectAgentProvider(credential=credential) as provider,
):
    try:
        # 1. Upload file and create vector store
        file_path = '../files/demo.md'
        print(f"Uploading file from: {file_path}")

        file = await agents_client.files.upload_and_poll(file_path=file_path, purpose="assistants")
        print(f"Uploaded file, file ID: {file.id}")

        vector_store = await agents_client.vector_stores.create_and_poll(file_ids=[file.id], name="contoso_knowledge_base")
        print(f"Created vector store, vector store ID: {vector_store.id}")

        # 2. Create file search tool with uploaded resources
        file_search_tool = HostedFileSearchTool(inputs=[Content.from_hosted_vector_store(vector_store_id=vector_store.id)])

        # 3. Create an agent with file search capabilities using the provider
        agent = await provider.create_agent(
            name="PythonRAGAgent",
            instructions=(
                "You are an AI assistant designed to answer user questions using only the information "
                "retrieved from the provided document(s). "
                "If a user's question cannot be answered using the retrieved context, you must clearly respond: "
                "'I'm sorry, but the uploaded document does not contain the necessary information to answer that question.' "
                "Do not answer from general knowledge or reasoning. Do not make assumptions or generate hypothetical explanations. "
                "For questions that do have relevant content in the document, respond accurately and cite the document explicitly."
            ),
            tools=file_search_tool,
        )

        # 4. Query the agent
        print(f"\n# User: '{query}'")
        response = await agent.run(query)
        print(f"# Agent: {response.text}")
        
    finally:
        # 5. Cleanup: Delete the vector store and file to prevent orphaned resources
        if vector_store:
            await agents_client.vector_stores.delete(vector_store.id)
            print(f"\nDeleted vector store: {vector_store.id}")
        if file:
            await agents_client.files.delete(file.id)
            print(f"Deleted file: {file.id}")

Uploading file from: ../files/demo.md
Uploaded file, file ID: assistant-VJjyysjZgRUErwWhvDLrxa
Created vector store, vector store ID: vs_vPxVd9hyk4uSAh1WpsIhDAjq

# User: 'What's GraphRAG?'
# Agent: GraphRAG is an AI-based content interpretation and search capability. It uses large language models (LLMs) to parse data, create a knowledge graph, and answer user questions about a user-provided private dataset. GraphRAG connects information across large volumes of data and leverages these connections to answer questions that are difficult or impossible to solve using traditional keyword or vector-based search methods. It supports queries where answers span multiple documents or where thematic insights, such as top themes in a dataset, are needed.

GraphRAG is intended for use in critical information discovery and analysis scenarios, especially where data may be noisy, contain misinformation, or where questions are abstract or thematic. It is designed for users experienced in responsible a

## 🚀 Create and Run the RAG Agent

This cell performs the complete RAG workflow with modern best practices:

**Workflow Steps:**
1. **Upload Document**: Upload the document file to Azure AI Foundry
2. **Create Vector Store**: Build a searchable vector index from the document
3. **Configure File Search Tool**: Set up the tool with the vector store reference
4. **Create Agent**: Use `AzureAIProjectAgentProvider` for simplified agent creation
5. **Query Agent**: Run the query and get grounded responses
6. **Cleanup Resources**: Automatically delete vector store and file to prevent resource leaks

**Key Improvements:**
- Uses `AzureAIProjectAgentProvider` for cleaner agent creation
- Uses `AgentsClient` for direct file and vector store operations
- Implements proper resource cleanup in `finally` block
- Simpler API with `agent.run()` instead of streaming
- Better error handling and resource management